In [ ]:
import cv2
import time

# Constants for speed calculation
PIXELS_PER_INCH = 10  # Assuming 10 pixels per inch
INCHES_TO_FEET = 1 / 12  # Convert inches to feet
MIN_CONTOUR_AREA = 1000  # Minimum contour area to be considered as a vehicle

def calculate_speed(distance_in_pixels, time_difference):
    distance_in_feet = distance_in_pixels / PIXELS_PER_INCH * INCHES_TO_FEET
    if time_difference == 0:
        return 0
    speed = distance_in_feet / time_difference
    return round(speed, 2)

# Function to detect vehicles using background subtraction
def detect_vehicles(frame, bg_subtractor, previous_center, previous_frame_time):
    # Apply background subtraction
    fg_mask = bg_subtractor.apply(frame)

    # Remove noise
    fg_mask = cv2.medianBlur(fg_mask, 5)

    # Find contours
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize current_time and center
    current_time = time.time()
    center = None

    # Draw rectangles around detected vehicles and calculate speed
    for contour in contours:
        if cv2.contourArea(contour) > MIN_CONTOUR_AREA:  # Filter out small contours
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Calculate center of the bounding box
            center = (x + w // 2, y + h // 2)
            if previous_center is not None:
                distance_in_pixels = abs(center[0] - previous_center[0])

                # Calculate time taken for vehicle to move the distance
                current_time = time.time()
                time_difference = current_time - previous_frame_time

                # Calculate speed of the vehicle
                speed = calculate_speed(distance_in_pixels, time_difference)

                # Display speed on the frame
                cv2.putText(frame, f'Speed: {speed} Km/hr', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame, center, current_time

# Open video file
video_path = 'highway.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Initialize background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Initialize variables for tracking
previous_center = None
previous_frame_time = time.time()  # Initialize previous_frame_time with current time

while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check if the frame is valid
    if not ret:
        break

    # Detect vehicles in the frame and get their speeds
    frame_with_rect, previous_center, previous_frame_time = detect_vehicles(frame, bg_subtractor, previous_center, previous_frame_time)

    # Display frame with rectangles drawn around vehicles
    cv2.imshow('Vehicles Detected', frame_with_rect)

    # Press 'q' to exit the program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture
cap.release()
cv2.destroyAllWindows()
